In [11]:
import pandas as pd
import scipy.io
import os
import mne
from mne.channels import make_standard_montage
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

In [12]:
data_dir = os.path.dirname("./data/")
data_files = os.listdir(data_dir)
print('\n'.join(sorted(data_files)))

FREEFORMSubjectB1511112StLRHand.mat
FREEFORMSubjectC1512082StLRHand.mat
FREEFORMSubjectC1512102StLRHand.mat
NoMTSubjectF1604220StNoMotor.mat
NoMTSubjectH1606280StNoMotor.mat
NoMTSubjectI1605120StNoMotor.mat
NoMTSubjectJ1610260StNoMotor.mat
NoMTSubjectK1610250StNoMotor.mat
NoMTSubjectL1610260StNoMotor.mat
NoMTSubjectM1611160StNoMotor.mat


In [13]:
def annotations_from_eGUI(raw, egui):
    codes = []
    starts = []
    
    current_state = None
    
    for i in range(len(egui)):
        if egui[i][0] != current_state:
            starts.append(i)
            current_state = egui[i][0]
            codes.append(str(egui[i][0]))
    
    starts.append(len(egui))
    codes = np.array(codes)
    sf = raw.info.get('sfreq')
    starts = np.array(starts) / sf
    durations = starts[1:] - starts[:-1]
    starts = starts[:-1]
    
    raw.set_annotations(mne.Annotations(onset=starts, duration=durations, description=codes))
            

def raw_from_mat(file):
    mat = scipy.io.loadmat(os.path.join(data_dir, file))
    
    sampling_freq = mat["o"][0][0][2][0][0]
    n_samples = mat["o"][0][0][3][0][0]
    ch_names = [element[0][0] for element in mat["o"][0][0][6]]
    
    df = pd.DataFrame(mat["o"][0][0][5], columns=ch_names)
    df = df.drop(columns=["X5"])
    df = df.T
    ch_names.remove("X5")
    
    ch_types = ['eeg'] * 21
    info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
    raw = mne.io.RawArray(df.to_numpy(), info)
    
    montage = make_standard_montage("standard_prefixed")
    raw.set_montage(montage)
    
    raw.load_data().set_eeg_reference(ref_channels='average')
    annotations_from_eGUI(raw, mat["o"][0][0][4])
    return raw


def process_raw(raw):
    raw.load_data().filter(0.1, 30, method="fir", phase="zero-double")

In [14]:
# files_5f = [file for file in data_files if "5F" in file]
raw_NoMT = [raw_from_mat(file) for file in data_files if "NoMT" in file]
# files_HaLT = [file for file in data_files if "HaLT" in file]
raw_FREEFORM = [raw_from_mat(file) for file in data_files if "FREEFORM" in file]
# files_CLA = [file for file in data_files if ("CLA" in file) and (not "SubjectJ" in file)]
# files_CLA_SubjectJ = [file for file in data_files if "CLA-" in file]

Creating RawArray with float64 data, n_channels=21, n_times=664600
    Range : 0 ... 664599 =      0.000 ...  3322.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=21, n_times=678400
    Range : 0 ... 678399 =      0.000 ...  3391.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=21, n_times=667600
    Range : 0 ... 667599 =      0.000 ...  3337.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=21, n_times=734400
    Range : 0 ... 734399 =      0.000 ...  3671.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 d

In [15]:
for file in raw_FREEFORM:
    print(Counter(file.annotations.description))

Counter({'0': 741, '1': 370, '2': 369, '99': 1})
Counter({'0': 705, '1': 350, '2': 350, '99': 4})
Counter({'0': 692, '1': 346, '2': 342, '99': 3})


In [16]:
for file in raw_NoMT:
    print(Counter(file.annotations.description))

Counter({'0': 960, '5': 169, '1': 161, '2': 157, '3': 157, '4': 155, '6': 152, '99': 5, '91': 2, '92': 1})
Counter({'0': 967, '5': 171, '1': 163, '2': 159, '3': 158, '4': 156, '6': 151, '99': 5, '91': 2, '92': 1})
Counter({'0': 968, '5': 171, '1': 163, '2': 159, '3': 158, '4': 156, '6': 152, '99': 5, '91': 2, '92': 1})
Counter({'0': 968, '5': 170, '1': 163, '2': 159, '3': 158, '4': 156, '6': 153, '99': 5, '91': 2, '92': 1})
Counter({'0': 968, '4': 180, '6': 163, '2': 163, '3': 160, '5': 155, '1': 139, '99': 5, '91': 1, '92': 1})
Counter({'0': 963, '5': 171, '1': 162, '2': 158, '3': 156, '4': 154, '6': 153, '99': 5, '91': 2, '92': 1})
Counter({'0': 966, '5': 171, '1': 163, '2': 159, '4': 156, '3': 156, '6': 152, '99': 5, '91': 2, '92': 1})


In [17]:
for file in raw_FREEFORM:
    process_raw(file)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-12 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-12 dB cutoff frequency: 33.75 Hz)
- Filter length: 6601 samples (33.005 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-12 dB c

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-12 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-12 dB cutoff frequency: 33.75 Hz)
- Filter length: 6601 samples (33.005 sec)



[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.1s finished


In [18]:
raw_FREEFORM[0].plot(scalings='auto')
plt.show()

Using pyopengl with version 3.1.6


1   HIToolbox                           0x00000001920145c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x0000000192014188 _ZN15MenuBarInstance14EnableAutoShowEv + 60
3   HIToolbox                           0x0000000191fb7310 SetMenuBarObscured + 372
4   HIToolbox                           0x0000000191fb6ee8 _ZN13HIApplication15HandleActivatedEP14OpaqueEventRefhP15OpaqueWindowPtrh + 172
5   HIToolbox                           0x0000000191fb0fcc _ZN13HIApplication13EventObserverEjP14OpaqueEventRefPv + 296
6   HIToolbox                           0x0000000191f77cd0 _NotifyEventLoopObservers + 176
7   HIToolbox                           0x0000000191fb096c AcquireEventFromQueue + 432
8   HIToolbox                           0x0000000191f9fc84 ReceiveNextEventCommon + 320
9   HIToolbox                           0x0000000191f9fb2c _BlockUntilNextEventMatchingListInModeWithFilter + 72
10  AppKit                              0x000000018bb4c424 _DPS

1   HIToolbox                           0x00000001920145c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x0000000191fde478 _ZL17BroadcastInternaljPvh + 184
3   SkyLight                            0x000000018d4f44dc _ZN12_GLOBAL__N_123notify_datagram_handlerEj15CGSDatagramTypePvmS1_ + 896
4   SkyLight                            0x000000018d8204c4 _ZN21CGSDatagramReadStream26dispatchMainQueueDatagramsEv + 228
5   SkyLight                            0x000000018d8203c0 ___ZN21CGSDatagramReadStream15mainQueueWakeupEv_block_invoke + 28
6   libdispatch.dylib                   0x00000001886649dc _dispatch_call_block_and_release + 32
7   libdispatch.dylib                   0x0000000188666504 _dispatch_client_callout + 20
8   libdispatch.dylib                   0x0000000188674d1c _dispatch_main_queue_drain + 928
9   libdispatch.dylib                   0x000000018867496c _dispatch_main_queue_callback_4CF + 44
10  CoreFoundation                      

1   HIToolbox                           0x00000001920145c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x0000000191fde478 _ZL17BroadcastInternaljPvh + 184
3   SkyLight                            0x000000018d4f44dc _ZN12_GLOBAL__N_123notify_datagram_handlerEj15CGSDatagramTypePvmS1_ + 896
4   SkyLight                            0x000000018d8204c4 _ZN21CGSDatagramReadStream26dispatchMainQueueDatagramsEv + 228
5   SkyLight                            0x000000018d8203c0 ___ZN21CGSDatagramReadStream15mainQueueWakeupEv_block_invoke + 28
6   libdispatch.dylib                   0x00000001886649dc _dispatch_call_block_and_release + 32
7   libdispatch.dylib                   0x0000000188666504 _dispatch_client_callout + 20
8   libdispatch.dylib                   0x0000000188674d1c _dispatch_main_queue_drain + 928
9   libdispatch.dylib                   0x000000018867496c _dispatch_main_queue_callback_4CF + 44
10  CoreFoundation                      

1   HIToolbox                           0x00000001920145c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x0000000191fde478 _ZL17BroadcastInternaljPvh + 184
3   SkyLight                            0x000000018d4f44dc _ZN12_GLOBAL__N_123notify_datagram_handlerEj15CGSDatagramTypePvmS1_ + 896
4   SkyLight                            0x000000018d8204c4 _ZN21CGSDatagramReadStream26dispatchMainQueueDatagramsEv + 228
5   SkyLight                            0x000000018d8203c0 ___ZN21CGSDatagramReadStream15mainQueueWakeupEv_block_invoke + 28
6   libdispatch.dylib                   0x00000001886649dc _dispatch_call_block_and_release + 32
7   libdispatch.dylib                   0x0000000188666504 _dispatch_client_callout + 20
8   libdispatch.dylib                   0x0000000188674d1c _dispatch_main_queue_drain + 928
9   libdispatch.dylib                   0x000000018867496c _dispatch_main_queue_callback_4CF + 44
10  CoreFoundation                      

...

1   HIToolbox                           0x00000001920145c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x0000000191fde478 _ZL17BroadcastInternaljPvh + 184
3   SkyLight                            0x000000018d4f44dc _ZN12_GLOBAL__N_123notify_datagram_handlerEj15CGSDatagramTypePvmS1_ + 896
4   SkyLight                            0x000000018d8204c4 _ZN21CGSDatagramReadStream26dispatchMainQueueDatagramsEv + 228
5   SkyLight                            0x000000018d8203c0 ___ZN21CGSDatagramReadStream15mainQueueWakeupEv_block_invoke + 28
6   libdispatch.dylib                   0x00000001886649dc _dispatch_call_block_and_release + 32
7   libdispatch.dylib                   0x0000000188666504 _dispatch_client_callout + 20
8   libdispatch.dylib                   0x0000000188674d1c _dispatch_main_queue_drain + 928
9   libdispatch.dylib                   0x000000018867496c _dispatch_main_queue_callback_4CF + 44
10  CoreFoundation                      

1   HIToolbox                           0x00000001920145c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x0000000191fde478 _ZL17BroadcastInternaljPvh + 184
3   SkyLight                            0x000000018d4f44dc _ZN12_GLOBAL__N_123notify_datagram_handlerEj15CGSDatagramTypePvmS1_ + 896
4   SkyLight                            0x000000018d8204c4 _ZN21CGSDatagramReadStream26dispatchMainQueueDatagramsEv + 228
5   SkyLight                            0x000000018d8203c0 ___ZN21CGSDatagramReadStream15mainQueueWakeupEv_block_invoke + 28
6   libdispatch.dylib                   0x00000001886649dc _dispatch_call_block_and_release + 32
7   libdispatch.dylib                   0x0000000188666504 _dispatch_client_callout + 20
8   libdispatch.dylib                   0x0000000188674d1c _dispatch_main_queue_drain + 928
9   libdispatch.dylib                   0x000000018867496c _dispatch_main_queue_callback_4CF + 44
10  CoreFoundation                      

1   HIToolbox                           0x00000001920145c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x0000000191fde478 _ZL17BroadcastInternaljPvh + 184
3   SkyLight                            0x000000018d4f44dc _ZN12_GLOBAL__N_123notify_datagram_handlerEj15CGSDatagramTypePvmS1_ + 896
4   SkyLight                            0x000000018d8204c4 _ZN21CGSDatagramReadStream26dispatchMainQueueDatagramsEv + 228
5   SkyLight                            0x000000018d8203c0 ___ZN21CGSDatagramReadStream15mainQueueWakeupEv_block_invoke + 28
6   libdispatch.dylib                   0x00000001886649dc _dispatch_call_block_and_release + 32
7   libdispatch.dylib                   0x0000000188666504 _dispatch_client_callout + 20
8   libdispatch.dylib                   0x0000000188674d1c _dispatch_main_queue_drain + 928
9   libdispatch.dylib                   0x000000018867496c _dispatch_main_queue_callback_4CF + 44
10  CoreFoundation                      

Channels marked as bad:
none


1   HIToolbox                           0x0000000191f9f90c _ZN15MenuBarInstance22RemoveAutoShowObserverEv + 44
2   HIToolbox                           0x0000000191fde478 _ZL17BroadcastInternaljPvh + 184
3   SkyLight                            0x000000018d4f44dc _ZN12_GLOBAL__N_123notify_datagram_handlerEj15CGSDatagramTypePvmS1_ + 896
4   SkyLight                            0x000000018d8204c4 _ZN21CGSDatagramReadStream26dispatchMainQueueDatagramsEv + 228
5   SkyLight                            0x000000018d8203c0 ___ZN21CGSDatagramReadStream15mainQueueWakeupEv_block_invoke + 28
6   libdispatch.dylib                   0x00000001886649dc _dispatch_call_block_and_release + 32
7   libdispatch.dylib                   0x0000000188666504 _dispatch_client_callout + 20
8   libdispatch.dylib                   0x0000000188674d1c _dispatch_main_queue_drain + 928
9   libdispatch.dylib                   0x000000018867496c _dispatch_main_queue_callback_4CF + 44
10  CoreFoundation                      0